In [1]:
import glob
import numpy as np
import pandas as pd
import parselmouth 
import statistics


from parselmouth.praat import call
from scipy.stats.mstats import zscore
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [2]:
# Go through all the wave files in the folder and measure all the acoustics
start = 0.0
end = 2.0
timeStep = 0.0
unit= "Hertz"

In [3]:
# This is the function to measure source acoustics.
def measurePitch(voiceID, f0min, f0max, unit, start, end, timeStep):
    sound = parselmouth.Sound(voiceID) # read the sound
    duration = call(sound, "Get total duration") # duration
   
    #Extract Pitch Paramenters
    pitch = call(sound, "To Pitch", timeStep, f0min, f0max) #create a praat pitch object
    meanF0 = call(pitch, "Get mean", start, end, unit) # get mean pitch
    stdevF0 = call(pitch, "Get standard deviation", start ,end, unit) # get standard deviation
    minF0 = call(pitch, "Get minimum", start ,end, unit, "Parabolic") # get min
    maxF0 = call(pitch, "Get maximum", start ,end, unit, "Parabolic") # get max

    #This part measures harmonics to noise ratio
    # 0.01 60 0.1 4.5
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, f0min, 0.1, 1.0)
    meanHNR = call(harmonicity, "Get mean", start, end)
    stdevHNR = call(harmonicity, "Get standard deviation", start, end)

    #This part measures jitter & shimmer
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    
    localJitter = call(pointProcess, "Get jitter (local)", start, end, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", start, end, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", start, end, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", start, end, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", start, end, 0.0001, 0.02, 1.3)

    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", start, end, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", start, end, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", start, end, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", start, end, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", start, end, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", start, end, 0.0001, 0.02, 1.3, 1.6)
    
    return duration, meanF0, stdevF0, meanHNR, stdevHNR, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer

In [4]:
# This function measures formants using Formant Position formula
def measureFormants1(sound, wave_file,f0min,f0max,unit):
    sound = parselmouth.Sound(sound) # read the sound
    pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)
    numPoints = call(pointProcess, "Get number of points")

    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []
    
    # Measure formants only at glottal pulses
    for point in range(0, numPoints):
        point += 1
        t = call(pointProcess, "Get time from index", point)
        f1 = call(formants, "Get value at time", 1, t, unit, 'Linear')
        f2 = call(formants, "Get value at time", 2, t, unit, 'Linear')
        f3 = call(formants, "Get value at time", 3, t, unit, 'Linear')
        f4 = call(formants, "Get value at time", 4, t, unit, 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
        f3_list.append(f3)
        f4_list.append(f4)
    
    f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
    f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
    f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
    f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']
    
    # calculate mean formants across pulses
    if len(f1_list)!=0:
        f1_mean = statistics.mean(f1_list)
    else:
        f1_mean = 0
    if len(f2_list)!=0:
        f2_mean = statistics.mean(f2_list)
    else:
        f2_mean = 0
    if len(f3_list)!=0:
        f3_mean = statistics.mean(f3_list)
    else:
        f3_mean = 0
    if len(f4_list)!=0:
        f4_mean = statistics.mean(f4_list)
    else:
        f4_mean = 0
 
    
    # calculate median formants across pulses, this is what is used in all subsequent calcualtions
    # you can use mean if you want, just edit the code in the boxes below to replace median with mean
    if len(f1_list)!=0:
        f1_median = statistics.median(f1_list)
    else:
        f1_median = 0
    if len(f2_list)!=0:
        f2_median = statistics.median(f2_list)
    else:
        f2_median = 0
    if len(f3_list)!=0:
        f3_median = statistics.median(f3_list)
    else:
        f3_median = 0
    if len(f4_list)!=0:
        f4_median = statistics.median(f4_list)
    else:
        f4_median = 0

    
    return f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median

In [5]:
#9-Detection of Bulbar Involvement in Patients With Amyotrophic Lateral Sclerosis by Machine Learning Voice Analysis Diagnostic 
def measureFormants2(sound, wave_file,f0min,f0max,start,end,unit):
    
    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []
    
    dur10 = ((end - start)/10)
    midpoint = (start + (end - start)/2)
    start2 = midpoint - dur10
    end2 = midpoint + dur10

    formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)

    f1_mean = call(formants, "Get mean", 1, start2, end2, unit)
    f2_mean = call(formants, "Get mean", 2, start2, end2, unit)
    f3_mean = call(formants, "Get mean", 3, start2, end2, unit)
    f4_mean = call(formants, "Get mean", 4, start2, end2, unit)
   



    
    return f1_mean, f2_mean, f3_mean, f4_mean

In [6]:
def runPCA(df):
    # z-score the Jitter and Shimmer measurements
    measures = ['localJitter', 'localabsoluteJitter', 'rapJitter', 'ppq5Jitter', 'ddpJitter',
                'localShimmer', 'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 'apq11Shimmer', 'ddaShimmer']
    x = df.loc[:, measures].values
    x = StandardScaler().fit_transform(x)
    # PCA
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents, columns = ['JitterPCA', 'ShimmerPCA'])
    principalDf
    return principalDf

In [7]:
# create lists to put the results
file_list = []
duration_list = []
mean_F0_list = []
sd_F0_list = []
meanHNR_list = []
stdevHNR_list = []
localJitter_list = []
localabsoluteJitter_list = []
rapJitter_list = []
ppq5Jitter_list = []
ddpJitter_list = []
localShimmer_list = []
localdbShimmer_list = []
apq3Shimmer_list = []
aqpq5Shimmer_list = []
apq11Shimmer_list = []
ddaShimmer_list = []
f1_mean_list = []
f2_mean_list = []
f3_mean_list = []
f4_mean_list = []

sex_list = []
healthy_list = []

meta = pd.read_excel('../0-bd/SVD/META/SVD.xls', sheet_name='SVD')
ids = meta['ID'].tolist()
healthy = meta['Healthy'].tolist()
sex = meta['Sex'].tolist()
for wave_file in glob.glob("../0-bd/SVD/BD/PHRASE/*.wav"):
    
    x = wave_file.replace("../0-bd/SVD/BD/PHRASE", "")
    id = x.replace("-phrase.wav", "")
    id = id.replace("\\", "")
    print(id)
    placeId = ids.index(int(id))

    sex_list.append(0 if sex[placeId]=="m" else 1)
    healthy_list.append(0 if healthy[placeId]=="n" else 1)
    f0min = 60 if sex[placeId]=="m" else 100
    f0max = 300 if sex[placeId]=="m" else 500

    sound = parselmouth.Sound(wave_file)
    (duration, meanF0, stdevF0, meanHNR, stdevHNR, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, 
    localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer) = measurePitch(sound, f0min, f0max, unit, start, end, timeStep)
    (f1_mean, f2_mean, f3_mean, f4_mean) = measureFormants2(sound, wave_file,f0min,f0max,start,end,unit)
    
    file_list.append(id) # make an ID list
    duration_list.append(duration) # make duration list
    mean_F0_list.append(meanF0) # make a mean F0 list
    sd_F0_list.append(stdevF0) # make a sd F0 list
    meanHNR_list.append(meanHNR) #add HNR data
    stdevHNR_list.append(stdevHNR) #add HNR data
    
    # add raw jitter and shimmer measures
    localJitter_list.append(localJitter)
    localabsoluteJitter_list.append(localabsoluteJitter)
    rapJitter_list.append(rapJitter)
    ppq5Jitter_list.append(ppq5Jitter)
    ddpJitter_list.append(ddpJitter)
    localShimmer_list.append(localShimmer)
    localdbShimmer_list.append(localdbShimmer)
    apq3Shimmer_list.append(apq3Shimmer)
    aqpq5Shimmer_list.append(aqpq5Shimmer)
    apq11Shimmer_list.append(apq11Shimmer)
    ddaShimmer_list.append(ddaShimmer)
    
    # add the formant data
    f1_mean_list.append(f1_mean)
    f2_mean_list.append(f2_mean)
    f3_mean_list.append(f3_mean)
    f4_mean_list.append(f4_mean)
  


# Add the data to Pandas
features_dataframe_nan = pd.DataFrame(np.column_stack([sex_list, healthy_list,duration_list, mean_F0_list, sd_F0_list, meanHNR_list, stdevHNR_list, 
                                   localJitter_list, localabsoluteJitter_list, rapJitter_list, 
                                   ppq5Jitter_list, ddpJitter_list, localShimmer_list, 
                                   localdbShimmer_list, apq3Shimmer_list, aqpq5Shimmer_list, 
                                   apq11Shimmer_list, ddaShimmer_list, f1_mean_list, 
                                   f2_mean_list, f3_mean_list, f4_mean_list]),
                                   columns=['sex', 'healthy','duration','meanF0Hz', 'stdevF0Hz', 'meanHNR', 'stdevHNR', 
                                            'localJitter', 'localabsoluteJitter', 'rapJitter', 
                                            'ppq5Jitter', 'ddpJitter', 'localShimmer', 
                                            'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 
                                            'apq11Shimmer', 'ddaShimmer', 'f1_mean', 'f2_mean', 
                                            'f3_mean', 'f4_mean'])

features_dataframe = features_dataframe_nan.dropna(axis=0 , how='any')
pca_dataframe = runPCA(features_dataframe) # Run jitter and shimmer PCA
features_dataframe = pd.concat([features_dataframe,pca_dataframe], axis=1) # Add PCA data
features_dataframe.to_csv("X_nan.csv", index=False)

dataframe = pd.read_csv('X_nan.csv')
features_dataframe_no_nan = dataframe.dropna(axis=0 , how='any')
features_dataframe_no_nan.to_csv("X.csv", index=False)


1
10
100
1000
1002
1003
1004
1005
1006
1007
1008
1009
101
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
102
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
103
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
104
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
105
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
106
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
107
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
108
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
109
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
11
110
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
111
1110
1111
1112
1113
1115
1116
1117
1118
1119
112
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
113
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
114
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
115
1150
1151
1153
1155
1156
1157
1158
1159
116
1160
1161
1162
1163
1164
1165
1166
1167
1168
1169
117
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
118
1180
1181
1182
1183
1184
1185
1186
1

684
685
686
687
688
689
69
690
691
692
693
694
695
696
697
698
699
7
70
700
701
702
703
704
705
706
707
708
709
71
710
711
712
714
715
716
718
719
72
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
74
740
741
742
743
744
745
746
747
77
78
782
79
8
80
803
804
805
806
807
808
809
81
810
811
812
813
814
815
816
817
818
819
82
820
821
822
823
824
825
826
827
828
829
83
830
831
832
833
834
835
836
837
838
839
84
840
841
842
843
844
845
846
847
848
849
85
850
851
852
853
854
855
857
859
86
860
861
862
863
864
865
867
868
869
87
870
871
872
873
874
875
876
877
878
879
88
880
881
883
884
885
886
887
888
889
89
890
891
892
893
894
895
896
897
898
899
9
90
900
901
902
903
904
907
908
909
91
910
911
912
913
914
915
916
917
918
919
92
920
921
922
923
924
925
926
927
928
929
93
930
931
932
933
934
935
936
937
938
939
94
940
941
942
943
944
945
946
947
948
949
95
950
951
952
953
954
955
956
957
958
959
96
960
961
962
963
964
965
967
968
969
97
970
971
972
973
974
975
